# EfficientAD Training

- EfficientADモデルを単一カテゴリデータ(Normal)で学習します

### （事前）CUDA Version 確認

In [ ]:
import torch, platform
print("torch version :", torch.__version__)
print("cuda in torch :", torch.version.cuda)
print("cuda available:", torch.cuda.is_available())


## 1. 初期設定

- 処理する画像サイズ 「IMAGE_SIZE」 と、検出対象とする画像カテゴリ 「CATEGORY」を指定してください
- 作成された「RUN_DIR」に、学習済モデルが作成されます

In [ ]:
from pathlib import Path
from datetime import datetime, timezone, timedelta
from collections import defaultdict
import yaml
import optuna
import torch
import timm, itertools, re

from anomalib.data import Folder
from anomalib.models import Padim
from anomalib.engine import Engine
from pytorch_lightning.loggers import TensorBoardLogger

# -------- ユーザ設定項目 --------
IMAGE_SIZE       = 256                         # 画像サイズ(★変更対象)
IMAGE_THRESHOLD  = 0.5                         # 異常検出閾値(★変更対象)
CATEGORY         = "VisA_pipe_fryum"           # 検出対象のカテゴリ(★変更対象)
DATA_ROOT        = Path("/workspace/data")     # データフォルダ(tarin/test)
# -------------------------------

JST = timezone(timedelta(hours=9))
timestamp = datetime.now(JST).strftime('%Y%m%d_%H%M%S')       # 実行時のシステム日時
OUTPUT_DIR  = Path("/workspace/models") / "EfficientAD" / CATEGORY  # 学習済モデルの出力先
RUN_DIR   = OUTPUT_DIR / timestamp
(RUN_DIR / 'checkpoint').mkdir(parents=True, exist_ok=True)
(RUN_DIR / 'pytorch').mkdir(parents=True, exist_ok=True)
TEMP_DIR = RUN_DIR / "temp"
PARAM_DIR = RUN_DIR / "param"
LOG_DIR = RUN_DIR / "logs"

print('RUN DIR:', RUN_DIR)

# -------- DataModule定義 --------
def build_datamodule() -> Folder:
    return Folder(
        name=CATEGORY,
        root=DATA_ROOT,
        normal_dir=f"train/{CATEGORY}",             # 学習用正常データ
        abnormal_dir=f"test/{CATEGORY}/anomaly",    # テスト(評価)用異常データ (学習では使われない/Optuna探索では使われる)
        normal_test_dir=f"test/{CATEGORY}/normal",  # テスト(評価)用正常データ
        train_batch_size=1,                         # EfficientADでは、1固定 
        eval_batch_size=32,
        extensions=(
            ".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp",
            ".JPG", ".JPEG", ".PNG", ".BMP", ".TIF", ".TIFF", ".WEBP",
        ),
    )

## 2. ハイパーパラメータ探索 (Optuna)

- ここでは、ハイパーパラメータの自動探索を行います
- PatchCore は、本来完全教師なし学習ですが、Optuna 探索は検証用に異常ラベルを必要とする教師ありプロセスと言えます
- 探索を行わずに手動でハイパーパラメータを調整する場合は、ここをスキップして「3. 学習 」へ

In [ ]:
from lightning.pytorch.callbacks import ModelCheckpoint 
from anomalib.models import EfficientAd
from anomalib.engine import Engine
import optuna, torch, yaml
from PIL import ImageFile

# -------- GPU利用可否 --------
GPU_OK = torch.cuda.is_available()

ImageFile.LOAD_TRUNCATED_IMAGES = True      # 破損した画像も取込む(ロジックとして除去するのもあり)

# -------- サーチスペース(★変更対象) --------
N_TRIALS    = 3
MODEL_SIZES = ["small", "medium"]           # largeは無し
LR_RANGE    = (1e-4, 1e-3)

# ---------- EfficientAdラッパー ----------
class EfficientAdNoIter(EfficientAd):
    def on_save_checkpoint(self, checkpoint):
        # pickle 化不可能なオブジェクトを取り除く
        # NotImplementedError: ('{} cannot be pickled', '_SingleProcessDataLoaderIter') 対策        
        self.__dict__.pop("imagenet_iterator", None)
        return super().on_save_checkpoint(checkpoint)
    def on_load_checkpoint(self, checkpoint):
        pass
        
def objective(trial: optuna.Trial):
    model_size = trial.suggest_categorical("model_size", MODEL_SIZES)
    lr         = trial.suggest_float("learning_rate", *LR_RANGE, log=True)

    dm  = build_datamodule()                       # build_datamodule内で train_batch_size=1 が必須
    pre = EfficientAd.configure_pre_processor(image_size=(IMAGE_SIZE, IMAGE_SIZE))
    model = EfficientAdNoIter(                     # ラッパー利用
        model_size=model_size, 
        lr=lr, 
        pre_processor=pre
        #imagenet_dir="path/to/local/imagenette"  # 事前学習画像のDLを外したい場合に指定
    )
  
    # -------------- Engine -----------------
    engine = Engine(
        default_root_dir    = TEMP_DIR / "optuna",
        accelerator         = "gpu" if GPU_OK else "cpu",
        max_epochs          = 1,
        logger              = False,
        enable_progress_bar = False,
    )

    # ---------- GPU/CPUフォールバック ----------
    used_gpu = None
    for use_gpu in (GPU_OK, False):
        try:
            engine.fit(model=model, datamodule=dm)
            used_gpu = use_gpu
            break
        except RuntimeError as e:
            if "cudaGetDeviceCount" in str(e):
                print("⚠️ CUDA 初期化失敗 → CPU で再試行")
                engine._cache.args["accelerator"] = "cpu"
                continue
            raise

    trial.set_user_attr("used_gpu", used_gpu)
    if not used_gpu:
        print(f"Trial {trial.number}: CPU で実行 (GPU fallback)")

    # 取得できない場合は 0.0
    return engine.trainer.callback_metrics.get("image_AUROC",
                                               torch.tensor(0.)).item()
# -------- Optuna 実行 --------
TEMP_DIR.mkdir(parents=True, exist_ok=True)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=N_TRIALS)

print("BEST :", study.best_params)
print("AUROC:", study.best_value)

# -------- 結果保存 --------
PARAM_DIR.mkdir(exist_ok=True)

search_space = dict(
    model_size    = MODEL_SIZES,
    learning_rate = {"low": LR_RANGE[0], "high": LR_RANGE[1]},
)
yaml.safe_dump(search_space, open(PARAM_DIR / "search_space.yaml", "w"))

best = study.best_params.copy()
yaml.safe_dump(best, open(PARAM_DIR / "best_params.yaml", "w"))
study.trials_dataframe().to_csv(PARAM_DIR / "trials.csv", index=False)

# ---------- TEMP_DIR のクリーンアップ ----------
# Debugする場合は、コメントアウトしてください
import shutil, gc
if TEMP_DIR.exists():
    # Windows でハンドルが残ると削除に失敗することがあるので、念のため GC
    gc.collect()
    shutil.rmtree(TEMP_DIR, ignore_errors=True)


## 3. 学習

- 「2. ハイパーパラメータ探索(Optuna)」 を行った場合は、保存されたベストパラメータで学習します
- 自動探索を行わない行場合は、ハイパーパラメータ 「MANUAL_PARAMS」を手動で調整してください

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import LearningRateMonitor
import numpy as np, torch, yaml, types
from anomalib.models import EfficientAd
from anomalib.engine import Engine
from anomalib.deploy import ExportType

# -------- GPU 利用可否 --------
GPU_OK = torch.cuda.is_available()

# -------- 学習設定 --------
MAX_EPOCHS = 1  # ※固定

# -------- ★手動パラメータ（best_params.yaml が無い時に使用） --------
MANUAL_PARAMS = dict(
    model_size    = "medium",      # "small" or "medium"
    learning_rate = 1e-4,
)

# ---------- EfficientAdラッパー ----------
class EfficientAdNoIter(EfficientAd):
    def on_save_checkpoint(self, checkpoint):
        # pickle 化不可能なオブジェクトを取り除く
        # NotImplementedError: ('{} cannot be pickled', '_SingleProcessDataLoaderIter') 対策        
        self.__dict__.pop("imagenet_iterator", None)
        return super().on_save_checkpoint(checkpoint)
    def on_load_checkpoint(self, checkpoint):
        pass

# -------- best_params.yaml 読み込み --------
best_params_path = PARAM_DIR / "best_params.yaml"

if best_params_path.exists():
    cfg = yaml.safe_load(open(best_params_path))
    print("▶ Using best_params.yaml:", cfg)
else:
    cfg = MANUAL_PARAMS.copy()
    print("▶ Using manual params:", cfg)

# -------- Model & DataModule --------
dm  = build_datamodule()            # build_datamodule内で train_batch_size=1 が必須

# -------- モデル定義 --------
pre   = EfficientAd.configure_pre_processor(image_size=(IMAGE_SIZE, IMAGE_SIZE))
model = EfficientAdNoIter(
    model_size   = cfg["model_size"],
    lr           = cfg["learning_rate"],
    pre_processor= pre,
)
# -------- Lightning学習 (GPU/CPUフォールバック) --------
tb_logger = TensorBoardLogger(save_dir=RUN_DIR / "logs", name="final")
used_gpu  = None
for use_gpu in (GPU_OK, False):
    try:
        engine = Engine(
            default_root_dir   = TEMP_DIR / "train",
            accelerator        = "gpu" if use_gpu else "cpu",
            max_epochs         = MAX_EPOCHS,
            log_every_n_steps  = 1,
            enable_progress_bar= False,
            logger             = tb_logger,
        )
        engine.fit(model=model, datamodule=dm)
        used_gpu = use_gpu
        break
    except RuntimeError as e:
        if "cudaGetDeviceCount" in str(e):
            warnings.warn("⚠️ CUDA 初期化エラー → CPU で再試行")
            continue
        raise
if not used_gpu:
    print("⚠️ GPU 使用不可 → CPU で学習完了")

# -------- Checkpoint 保存 --------
ckpt_path = RUN_DIR / "checkpoint" / "best.ckpt"
engine.trainer.save_checkpoint(ckpt_path)
print("✓ Checkpoint :", ckpt_path)

# -------- 推論用モデル .pth 保存 -------- 
state_path = RUN_DIR / "pytorch" / "model.pth"
torch.save(model.state_dict(), state_path)
print("✓ Weights :", state_path)

# ======== 学習データのみ推論して image_min, image_max を取得 ========
# 正規化のために推論処理へ連携する (別の方法 Z-Score(μ, σ)正規化もあり)
print(f"Computing image_min and image_max ...")

# Post-Processor の正規化だけオフにする
model.post_processor.enable_normalization = False

# DataModule を train のみでセットアップ
dm.setup("fit")                         # train フェーズ相当
predict_loader = dm.train_dataloader()  # train データを予測

# Engine.predict で生マップを取得
scores = []
logging.getLogger("anomalib.visualization.image.item_visualizer").setLevel(logging.ERROR)
for batch in engine.predict(model=model, datamodule=dm, return_predictions=True):
    # batch は list of ImagePrediction
    for item in batch:
        # 生の anomaly_map を最大値で画像ごとの raw スコアに
        raw = float(item.anomaly_map.max().cpu())
        scores.append(raw)

# 正規化機構を元に戻す
model.post_processor.enable_normalization = True

# image_min と image_max を計算
image_min = float(np.min(scores))
image_max = float(np.max(scores))
print(f"Computed image_min: {image_min}, image_max: {image_max}")
# ==================================================================

# ------- .pth に含まれないメタ情報を出力(json) -------
import json

meta_path = RUN_DIR / "pytorch" / "meta.json"
meta = {
    "model_size"          : cfg["model_size"],
    "learning_rate"       : cfg["learning_rate"],
    "image_size"          : IMAGE_SIZE,
    "weights_pth"         : state_path.name,
    "image_threshold"     : IMAGE_THRESHOLD,  # 手動設定
    "image_threshold_auto": IMAGE_THRESHOLD,  # Test後に上書き用(任意)
    "image_min"       : image_min,
    "image_max"       : image_max,    
}

with open(meta_path, "w", encoding="utf-8") as f:
    json.dump(meta, f, indent=2, ensure_ascii=False)

print("✓ Meta JSON :", meta_path)

# ---------- TEMP_DIR のクリーンアップ ----------
# Debugする場合は、コメントアウトしてください。
import shutil, gc
if TEMP_DIR.exists():
    # Windows でハンドルが残ると削除に失敗することがあるので、念のため GC
    gc.collect()
    shutil.rmtree(TEMP_DIR, ignore_errors=True)


## 4. 検出性能テスト

In [ ]:
from pathlib import Path
from anomalib.models import EfficientAd
from anomalib.engine import Engine
import torch, pandas as pd, shutil
import logging 

# ---------- パス・デバイス ----------
state_path  = Path(RUN_DIR / "pytorch" / "model.pth")
result_root = RUN_DIR / "test_result"
device      = "cuda" if torch.cuda.is_available() else "cpu"

# ---------- モデル ----------
pre   = EfficientAd.configure_pre_processor(image_size=(IMAGE_SIZE, IMAGE_SIZE))
model = EfficientAd(
    model_size   = cfg["model_size"],
    lr           = cfg["learning_rate"],
    pre_processor= pre,
).to(device)
model.load_state_dict(torch.load(state_path, map_location=device), strict=True)

# ---------- DataModule ----------
dm = build_datamodule(); dm.setup("test")

# suppress visualizer logs
logging.getLogger("anomalib.visualization.image.item_visualizer").setLevel(logging.ERROR)

# ---------- テスト (Visualizer はデフォルトのまま有効) ----------
engine = Engine(
    accelerator="gpu" if device == "cuda" else "cpu",
    devices=1,
    enable_progress_bar=False,
    logger=False,
)
engine.test(model=model, datamodule=dm)

metrics = engine.trainer.callback_metrics.copy()          # predict 前に退避
auroc   = float(metrics.get("image_AUROC", 0))
f1      = float(metrics.get("image_F1Score", 0))

# testで自動決定される image_threshold 
threshold = float(getattr(model.post_processor, "image_threshold", IMAGE_THRESHOLD))

# ---------- 可視化画像を result_root へ集約 ----------
# Visualizer は  results/EfficientAD/<CATEGORY>/<timestamp>/images/*
default_root = Path("results") / "EfficientAD" / CATEGORY
if default_root.exists():
    # 直近 (mtime が最大) の test_run を取得
    latest = max(default_root.iterdir(), key=lambda p: p.stat().st_mtime)
    src_images = latest / "images"                    # normal / anomaly
    dst_images = result_root / "images"
    for lbl_dir in src_images.glob("*"):              # normal & anomaly
        (dst_images / lbl_dir.name).mkdir(parents=True, exist_ok=True)
        for img in lbl_dir.glob("*"):
            shutil.copy2(img, dst_images / lbl_dir.name / img.name)

# ---------- 1枚ずつ結果を表示  ----------
def get_gt(item: "ImagePrediction") -> int:
    """0=normal, 1=anomaly を返す。属性が無ければパスで判定"""
    for key in ("label", "labels", "is_anomaly", "targets"):
        if hasattr(item, key):
            return int(getattr(item, key))

    # fallback
    parent = Path(item.image_path).parent.name.lower()
    return 1 if parent == "anomaly" else 0

records = []
for batch in engine.predict(model=model, datamodule=dm, return_predictions=True):
    for item in batch:
        file  = Path(item.image_path).name
        score = float(item.pred_score)
        pred  = "anomaly" if int(item.pred_label) == 1 else "normal"
        gt    = "anomaly" if get_gt(item) else "normal"
        print(f"{file:40s} | score={score:7.4f} | pred={pred:7s} | label={gt}")
        records.append(dict(file=file, score=score, pred=pred, label=gt))

# ---------- CSV 保存 ----------
result_root.mkdir(parents=True, exist_ok=True)
csv_path = result_root / "predictions.csv"
pd.DataFrame(records).to_csv(csv_path, index=False)
print(f"\n✓ predictions.csv saved to {csv_path}\n")

# ---------- meta.json 更新 (image_threshold_auto) ----------
meta_path = Path(RUN_DIR) / "pytorch" / "meta.json"
with open(meta_path, "r") as f:
    meta = json.load(f)
meta["image_threshold_auto"] = float(threshold)
with open(meta_path, "w") as f:
    json.dump(meta, f, indent=2, ensure_ascii=False)
print(f"✓ threshold_auto updated to {meta_path}")

# ---------- 指標 ----------
print("\n==========  EVALUATION  ==========")
print(f"Images tested : {len(dm.test_data)}")
print(f"AUROC         : {auroc:7.4f}")
print(f"Best F1       : {f1:7.4f}")
print("===================================\n")
